<a href="https://colab.research.google.com/github/silviootero/Trabajo-de-Programacion-Lineal/blob/main/M%C3%A9todo_Simplex_revisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PRIMER BOCETO DEL CÓDIGO**

In [ ]:
import numpy as np

class Simplex:
    def __init__(self, tableau):
        self.tableau = tableau

    def solve(self):
        # Implementar la lógica para resolver el problema de programación lineal utilizando el método simplex revisado
        pass

    def display_solution(self):
        # Implementar la lógica para mostrar la solución paso a paso
        pass

    def graphical_method(self):
        # Implementar el método gráfico para problemas de dos variables
        pass

    def sensitivity_analysis(self):
        # Implementar el análisis de sensibilidad
        pass

# Ejemplo de uso
if __name__ == "__main__":
    # Crear un objeto Simplex con el tableau del problema
    tableau = np.array([[1, -1, 1, 0, 0, 4],
                        [2,  3, 0, 1, 0, 12],
                        [3,  1, 0, 0, 1, 18],
                        [-4, -3, 0, 0, 0, 0]])  # El tableau debe estar en forma aumentada
    simplex_solver = Simplex(tableau)

    # Resolver el problema de programación lineal
    solution = simplex_solver.solve()

    # Mostrar la solución paso a paso
    simplex_solver.display_solution()

    # Realizar el método gráfico (solo para problemas con dos variables)
    simplex_solver.graphical_method()

    # Realizar el análisis de sensibilidad
    simplex_solver.sensitivity_analysis()


**FORMULACIÓN DEL MÉTODO SIMPLEX**

In [ ]:
def crear_matriz_simplex_aumentada(datos):
    # Transforma el problema a la forma estándar y crea la matriz simplex aumentada

def identificar_variable_pivote_avanzada(matriz_simplex):
    # Considera variables artificiales y de holgura para seleccionar la variable pivote

def verificar_optimalidad_avanzada(matriz_simplex):
    # Verifica si la solución es óptima considerando variables artificiales y de holgura

# Lectura y transformación del problema
datos = leer_datos_problema()
matriz_simplex_aumentada = crear_matriz_simplex_aumentada(datos)

# Iteración del método simplex
while True:
    var_entrada, var_salida = identificar_variable_pivote_avanzada(matriz_simplex_aumentada)

    if verificar_optimalidad_avanzada(matriz_simplex_aumentada):
        break

    realizar_pivotacion(matriz_simplex_aumentada, var_entrada, var_salida)

# Mostrar la solución óptima
mostrar_solucion(matriz_simplex_aumentada)


EJEMPLO CORTO

In [2]:
import numpy as np
from scipy.optimize import linprog

# Definición del problema de programación lineal
c = [-2, -3]  # Coeficientes de la función objetivo a minimizar
A_eq = [[1, 1]]  # Coeficientes de las restricciones de igualdad
b_eq = [4]  # Límites de las restricciones de igualdad
A_ub = [[2, 1]]  # Coeficientes de las restricciones de desigualdad
b_ub = [5]  # Límites de las restricciones de desigualdad

# Resolución del problema utilizando el método simplex revisado
res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, method='highs')
print("Solución utilizando método Simplex Revisado:")
print(res)


Solución utilizando método Simplex Revisado:
        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -12.0
              x: [ 0.000e+00  4.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  4.000e+00]
                 marginals: [ 1.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00]
                 marginals: [-3.000e+00]
        ineqlin:  residual: [ 1.000e+00]
                 marginals: [-0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0


**ESTRUCTURA MEJOR ESTABLECIDA**

Z = 3x1 + 2x2


*   2x1 + x2 ≤ 4
*   x1 + 2x2 ≤ 6
*   x1 ≥ 0
*   x2 ≥ 0

In [12]:
import numpy as np
from scipy.optimize import linprog

# Coeficientes de la función objetivo (negativos para convertir el problema de maximización en minimización)
c = [-3, -2]

# Coeficientes de las restricciones de desigualdad
A_ub = [[2, 1], [1, 2]]

# Límites de las restricciones de desigualdad
b_ub = [4, 6]

# Límites de las variables (x1 >= 0, x2 >= 0)
x_bounds = (0, None)

# Resolución del problema utilizando el método simplex revisado
res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=(x_bounds, x_bounds), method='highs')
print("Solución utilizando método Simplex Revisado:")
print(res)


Solución utilizando método Simplex Revisado:
        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -7.333333333333334
              x: [ 6.667e-01  2.667e+00]
            nit: 2
          lower:  residual: [ 6.667e-01  2.667e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [-1.333e+00 -3.333e-01]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0


In [27]:
# Coeficientes de la función objetivo
c = np.array([3, 2])

# Matriz de coeficientes de las restricciones
A = np.array([[2, 1], [1, 2]])

# Términos independientes de las restricciones
b = np.array([4, 6])

In [28]:
def crear_matriz_simplex(A, b, c):
  """
  Crea la matriz simplex inicial a partir de los coeficientes de la función objetivo,
  las restricciones y las variables.

  Args:
    A: Matriz de coeficientes de las restricciones (m x n).
    b: Vector de términos independientes de las restricciones (m x 1).
    c: Vector de coeficientes de la función objetivo (n x 1).

  Returns:
    matriz_simplex: Matriz simplex inicial aumentada (m x n+1).
  """
  n = A.shape[1]  # Número de variables
  m = A.shape[0]  # Número de restricciones

  # Crea la matriz simplex aumentada
  matriz_simplex = np.zeros((m, n + 1))
  matriz_simplex[:, :n] = A
  matriz_simplex[:, n] = -b
  matriz_simplex = np.vstack((matriz_simplex, np.hstack((c, 0))))

  # Añadir variables artificiales (forma corregida)
  for i in range(m):
    if A[i, :].sum() == 0:
      matriz_simplex = np.vstack((matriz_simplex, np.hstack((np.zeros(n + 1), -1))))

  # Añadir variables de holgura (si es necesario)
  # ... (código para añadir variables de holgura)

  return matriz_simplex



In [ ]:
#EN CASO DE QUE LA DE ARRIBA FALLE ESTA ES UNA ALTERNATIVA
import numpy as np

def crear_matriz_simplex(A, b, c):
  """
  Crea la matriz simplex inicial a partir de los coeficientes de la función objetivo,
  las restricciones y las variables.

  Args:
    A: Matriz de coeficientes de las restricciones (m x n).
    b: Vector de términos independientes de las restricciones (m x 1).
    c: Vector de coeficientes de la función objetivo (n x 1).

  Returns:
    matriz_simplex: Matriz simplex inicial aumentada (m x n+1).
  """
  n = A.shape[1]  # Número de variables
  m = A.shape[0]  # Número de restricciones

  # Crea la matriz simplex aumentada
  matriz_simplex = np.zeros((m, n + 1))
  matriz_simplex[:, :n] = A
  matriz_simplex[:, n] = -b
  matriz_simplex = np.vstack((matriz_simplex, np.hstack((c, 0))))

  # Añadir variables artificiales (forma corregida)
  for i in range(m):
    if A[i, :].sum() == 0:
      matriz_simplex = np.vstack((matriz_simplex, np.hstack((np.zeros(n + 1), -1))))

  # Añadir variables de holgura (si es necesario)
  for i in


In [29]:
matriz_simplex = crear_matriz_simplex(A, b, c)
print(matriz_simplex)


[[ 2.  1. -4.]
 [ 1.  2. -6.]
 [ 3.  2.  0.]]


In [30]:
def identificar_variable_pivote(matriz_simplex):
  """
  Identifica la variable de entrada básica que debe salir de la base y la variable
  de no-entrada básica que debe entrar.

  Args:
    matriz_simplex: Matriz simplex actual (m x n+1).

  Returns:
    var_entrada: Índice de la variable de entrada básica que debe salir de la base.
    var_no_entrada: Índice de la variable de no-entrada básica que debe entrar.
  """
  # Obtiene el índice de la fila con el ratio de costo reducido más negativo
  var_no_entrada = np.argmin(matriz_simplex[-1, :-1])

  # Si todos los ratios de costo reducido son no positivos, ya se encontró la solución óptima
  if min(matriz_simplex[-1, :-1]) >= 0:
    return None, None

  # Identifica la variable de entrada básica que debe salir de la base
  ratio_min = np.inf
  var_entrada = None
  for i in range(matriz_simplex.shape[0] - 1):
    if matriz_simplex[i, var_no_entrada] > 0:
      ratio = matriz_simplex[i, -1] / matriz_simplex[i, var_no_entrada]
      if ratio < ratio_min:
        ratio_min = ratio
        var_entrada = i

  return var_entrada, var_no_entrada


In [31]:
def verificar_optimalidad(matriz_simplex):
  """
  Verifica si la solución actual es óptima.

  Args:
    matriz_simplex: Matriz simplex actual (m x n+1).

  Returns:
    True si la solución es óptima, False en caso contrario.
  """
  # Si todos los ratios de costo reducido son no positivos, ya se encontró la solución óptima
  if min(matriz_simplex[-1, :-1]) >= 0:
    return True
  else:
    return False


In [32]:
def realizar_pivotacion(matriz_simplex, var_entrada, var_no_entrada):
  """
  Realiza la operación de pivotación en el método simplex.

  Args:
    matriz_simplex: Matriz simplex actual (m x n+1).
    var_entrada: Índice de la variable de entrada básica que debe salir de la base.
    var_no_entrada: Índice de la variable de no-entrada básica que debe entrar en la base.
  """
  # Normaliza la fila del pivote (divide la fila por el elemento pivote)
  matriz_simplex[var_entrada] /= matriz_simplex[var_entrada, var_no_entrada]

  # Elimina los demás elementos en la columna de la variable de entrada
  for i in range(matriz_simplex.shape[0]):
    if i != var_entrada:
      matriz_simplex[i] -= matriz_simplex[i, var_no_entrada] * matriz_simplex[var_entrada]


In [33]:
while True:
  var_entrada, var_no_entrada = identificar_variable_pivote(matriz_simplex)

  if var_entrada is None:
    break

  realizar_pivotacion(matriz_simplex, var_entrada, var_no_entrada)
  print(matriz_simplex)

# Verificar la optimalidad
if verificar_optimalidad(matriz_simplex):
  print("Solución óptima encontrada.")
else:
  print("No se encontró una solución óptima.")


Solución óptima encontrada.


In [34]:
def mostrar_solucion(matriz_simplex):
  """
  Muestra la solución óptima del problema.

  Args:
    matriz_simplex: Matriz simplex final (m x n+1).
  """
  variables_basicas = []
  valores_basicos = []

  # Identificar las variables básicas y sus valores
  for i in range(matriz_simplex.shape[0]):
    if abs(matriz_simplex[i, -1]) > 1e-6:
      variables_basicas.append(i)
      valores_basicos.append(matriz_simplex[i, -1])

  # Mostrar la solución
  print("Variables básicas:", variables_basicas)
  print("Valores básicos:", valores_basicos)
  print("Valor óptimo de Z:", matriz_simplex[-1, -1])

mostrar_solucion(matriz_simplex)


Variables básicas: [0, 1]
Valores básicos: [-4.0, -6.0]
Valor óptimo de Z: 0.0
